In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math

import time

%matplotlib inline


from spacy.lang.en import English



import bz2
from collections import Counter
import re
from nltk.tokenize import word_tokenize
import nltk
import numpy as np
nltk.download('punkt')



[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Tsenguunn\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:

main_data = pd.read_csv( "C://Tsenguun/datasets/Lending club/LoanStats3a_securev1.csv", delimiter="," )

pd.set_option('display.max_columns', None)

data_1 = main_data[ [ 'loan_amnt', 'funded_amnt', 'term', 'int_rate', 'installment', \
                     'emp_title', 'emp_length', 'home_ownership', \
                     'annual_inc', 'verification_status', 'loan_status', 'desc', \
                     'zip_code', 'addr_state', 'delinq_2yrs', 'mths_since_last_delinq', \
                     'mths_since_last_record', 'total_pymnt', 'recoveries', 'last_pymnt_d', \
                     'last_pymnt_amnt', 'acc_now_delinq', 'chargeoff_within_12_mths', \
                     'delinq_amnt' ] ]


data_2 = data_1.loc[ ~( data_1['desc'].isnull() ) ].reset_index(drop=True)


data_2 = data_2[ data_2['loan_status'].isin([ 'Charged Off', 'Fully Paid', 'Does not meet the credit policy. Status:Charged Off', 'Does not meet the credit policy. Status:Fully Paid' ]) ]
data_2.reset_index(drop=True, inplace=True)

data_2['y_hat'] = 1
data_2.loc[ data_2['loan_status'].isin([ 'Charged Off', 'Does not meet the credit policy. Status:Charged Off']), 'y_hat' ] = 0

data_2.shape


'\nmain_data = pd.read_csv( "C://Tsenguun/datasets/Lending club/LoanStats3a_securev1.csv", delimiter="," )\n\npd.set_option(\'display.max_columns\', None)\n\ndata_1 = main_data[ [ \'loan_amnt\', \'funded_amnt\', \'term\', \'int_rate\', \'installment\',                      \'emp_title\', \'emp_length\', \'home_ownership\',                      \'annual_inc\', \'verification_status\', \'loan_status\', \'desc\',                      \'zip_code\', \'addr_state\', \'delinq_2yrs\', \'mths_since_last_delinq\',                      \'mths_since_last_record\', \'total_pymnt\', \'recoveries\', \'last_pymnt_d\',                      \'last_pymnt_amnt\', \'acc_now_delinq\', \'chargeoff_within_12_mths\',                      \'delinq_amnt\' ] ]\n\n\ndata_2 = data_1.loc[ ~( data_1[\'desc\'].isnull() ) ].reset_index(drop=True)\n\n\ndata_2 = data_2[ data_2[\'loan_status\'].isin([ \'Charged Off\', \'Fully Paid\', \'Does not meet the credit policy. Status:Charged Off\', \'Does not meet the credit polic

In [3]:
data_3 = data_2[['loan_amnt', 'funded_amnt', 'term', 'int_rate', 'installment',
       'emp_length', 'home_ownership', 'annual_inc',
       'verification_status', 'loan_status', 'desc', 'zip_code', 'addr_state',
       'delinq_2yrs', 'total_pymnt', 'recoveries', 'last_pymnt_d', 'last_pymnt_amnt',
       'acc_now_delinq', 'chargeoff_within_12_mths', 'delinq_amnt', 'y_hat']]

data_3.dropna(inplace=True)

missing_props_2 = data_3.isna().sum() / len(data_3)
missing_props_2.sort_values(ascending=False)

C:\Users\Tsenguunn\AppData\Local\Temp\ipykernel_179936\2830042887.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_3.dropna(inplace=True)


loan_amnt                   0.0
funded_amnt                 0.0
delinq_amnt                 0.0
chargeoff_within_12_mths    0.0
acc_now_delinq              0.0
last_pymnt_amnt             0.0
last_pymnt_d                0.0
recoveries                  0.0
total_pymnt                 0.0
delinq_2yrs                 0.0
addr_state                  0.0
zip_code                    0.0
desc                        0.0
loan_status                 0.0
verification_status         0.0
annual_inc                  0.0
home_ownership              0.0
emp_length                  0.0
installment                 0.0
int_rate                    0.0
term                        0.0
y_hat                       0.0
dtype: float64

In [4]:
data_3 = data_3.sample(frac=1).reset_index(drop=True)

In [5]:
data_4 = pd.get_dummies(data_3, columns=['term'], prefix='term', dtype=float)

data_4['int_rate_num'] = data_4.apply( lambda x: float( x['int_rate'].replace("%", "") ) / 100, axis=1 )
data_4.drop(columns=['int_rate'], inplace=True)

data_4 = pd.get_dummies(data_4, columns=['emp_length'], prefix='emp_length', dtype=float)
data_4 = pd.get_dummies(data_4, columns=['home_ownership'], prefix='home_ownership', dtype=float)
data_4 = pd.get_dummies(data_4, columns=['verification_status'], prefix='verification_status', dtype=float)

data_4.drop(columns=['loan_status'], inplace=True)
data_4.drop(columns=['zip_code'], inplace=True)
data_4.drop(columns=['addr_state'], inplace=True)
data_4.drop(columns=['last_pymnt_d'], inplace=True)

data_4['y_hat'] = data_4['y_hat'].astype(float)


In [6]:
data_4.dtypes

loan_amnt                              float64
funded_amnt                            float64
installment                            float64
annual_inc                             float64
desc                                    object
delinq_2yrs                            float64
total_pymnt                            float64
recoveries                             float64
last_pymnt_amnt                        float64
acc_now_delinq                         float64
chargeoff_within_12_mths               float64
delinq_amnt                            float64
y_hat                                  float64
term_ 36 months                        float64
term_ 60 months                        float64
int_rate_num                           float64
emp_length_1 year                      float64
emp_length_10+ years                   float64
emp_length_2 years                     float64
emp_length_3 years                     float64
emp_length_4 years                     float64
emp_length_5 

In [7]:
data_len = data_4.shape[0]
test_len = np.int(np.floor(data_len*0.15))
val_len = np.int(np.floor(data_len*0.15))
train_len = data_len - test_len - val_len

print(f"val_len: {val_len}, test_len: {test_len}, train_len: {train_len}")

val_len: 4218, test_len: 4218, train_len: 19688


C:\Users\Tsenguunn\AppData\Local\Temp\ipykernel_179936\4048907440.py:2: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  test_len = np.int(np.floor(data_len*0.15))
C:\Users\Tsenguunn\AppData\Local\Temp\ipykernel_179936\4048907440.py:3: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check

In [8]:
nlp = English()

In [9]:
tr_dat = data_4.iloc[0:train_len, :]
tr_dat.shape

(19688, 35)

In [10]:
val_dat = data_4.iloc[train_len:train_len+val_len, :]
val_dat.shape

(4218, 35)

In [11]:
te_dat = data_4.iloc[train_len+val_len:, :]
te_dat.shape

(4218, 35)

In [12]:
def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        return False

In [13]:
#del words

len_tr = tr_dat.shape[0]
words = Counter()
desc_words_tr = []

In [14]:
for i in range(len_tr):
    
    
    token_list = []
    ind = -1
    
    test1 = nlp(tr_dat['desc'][i])
    
    for i, token in enumerate(test1):
        if i >= 5:
            tmp = token.text
            tmp = tmp.replace("<br", "")
            tmp = tmp.replace("br/", "")
            tmp = tmp.replace("/", "")
            tmp = tmp.replace(">", "")
            tmp = tmp.replace("%", "")
            if ( not is_number(tmp) ) and (tmp != '') and (tmp != ' ') and (tmp != '  ') and (tmp != '<') and (tmp != 'br') and (tmp != '/') and (tmp != '(') and (tmp != ')') and (tmp != '&') and (tmp != '*'):
            
                if tmp == "Borrower":
                    ind = i + 3
                if  i > ind:
                    if ( (tmp[-1] == ".") and (len(tmp) >=2) ):
                        token_list.extend( [tmp[:-1].lower(), tmp[-1]] )
                        word = tmp[:-1]
                        words.update( [word.lower()] )
                    else:
                        token_list.append( tmp.lower() )
                        word = tmp
                        words.update( [word.lower()] )
                        
    desc_words_tr.append(token_list)

In [15]:
# Removing the words that only appear once
words = {k:v for k,v in words.items() if v>1}

In [16]:
len(words)

12389

In [17]:
# Sorting the words according to the number of appearances, with the most common word being first
words = sorted(words, key=words.get, reverse=True)

In [18]:
# Adding padding and unknown to our vocabulary so that they will be assigned an index
words = ['_PAD','_UNK'] + words

In [19]:
#del word2idx, idx2word

word2idx = {o:i for i,o in enumerate(words)}
idx2word = {i:o for i,o in enumerate(words)}

In [20]:
#desc_words_tr[19]

In [21]:
tr_dat['desc_len'] = 0

C:\Users\Tsenguunn\AppData\Local\Temp\ipykernel_179936\3383454956.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tr_dat['desc_len'] = 0


In [22]:
for i, sentence in enumerate(desc_words_tr):
    # Looking up the mapping dictionary and assigning the index to the respective words
    desc_words_tr[i] = [word2idx[word] if word in word2idx else 1 for word in sentence]
    tr_dat.loc[i, 'desc_len'] = len(sentence)

In [23]:
tr_dat['desc_len'] = tr_dat['desc_len'].astype(float)

C:\Users\Tsenguunn\AppData\Local\Temp\ipykernel_179936\54064017.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tr_dat['desc_len'] = tr_dat['desc_len'].astype(float)


In [24]:
# Defining a function that either shortens sentences or pads sentences with 0 to a fixed length
def pad_input(sentences, seq_len):
    features = np.zeros((len(sentences), seq_len),dtype=int)
    for ii, review in enumerate(sentences):
        if len(review) != 0:
            features[ii, -len(review):] = np.array(review)[:seq_len]
    return features

In [25]:
seq_len = 300

In [26]:
desc_words_tr = pad_input(desc_words_tr, seq_len)

In [27]:
[idx2word[ww] for ww in desc_words_tr[2][:]]

['_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 

In [28]:
tr_dat.loc[2, 'desc']

'  Borrower added on 09/22/11 > After a rough few years financially I have a accumulated some credit card debt. Now that my job and monthly financials are stable, I would like to pay off all the cards with a lower interest loan so that I can leave them behind and make my way out of debt.<br/>null'

In [29]:
type(desc_words_tr)

numpy.ndarray

In [30]:
train_num_data = tr_dat.drop(columns=['desc'], inplace=False)

In [31]:
train_y = train_num_data['y_hat']

In [32]:
train_num_data.drop(columns=['y_hat'], inplace=True)

In [33]:
train_num_data.dtypes

loan_amnt                              float64
funded_amnt                            float64
installment                            float64
annual_inc                             float64
delinq_2yrs                            float64
total_pymnt                            float64
recoveries                             float64
last_pymnt_amnt                        float64
acc_now_delinq                         float64
chargeoff_within_12_mths               float64
delinq_amnt                            float64
term_ 36 months                        float64
term_ 60 months                        float64
int_rate_num                           float64
emp_length_1 year                      float64
emp_length_10+ years                   float64
emp_length_2 years                     float64
emp_length_3 years                     float64
emp_length_4 years                     float64
emp_length_5 years                     float64
emp_length_6 years                     float64
emp_length_7 

In [34]:
train_y = np.array(train_y)
type(train_y)

numpy.ndarray

In [35]:
train_y.shape

(19688,)

In [36]:
train_num_data = np.array(train_num_data)
type(train_num_data)

numpy.ndarray

In [37]:
train_num_data.dtype

dtype('float64')

In [38]:
np.savetxt( 'desc_words_tr.csv', desc_words_tr, delimiter=',' )

In [39]:
np.savetxt( 'train_num_data.csv', train_num_data, delimiter=',' )

In [40]:
np.savetxt( 'train_y.csv', train_y, delimiter=',' )

In [41]:
import os
os.getcwd()

'C:\\Users\\Tsenguunn'

In [42]:
glove_path = "C:/Users/Tsenguunn/Documents/Kaggle/pretrained embeddings glove"

In [43]:
embedding_dict = {}

with open(f"{glove_path}/glove.6B.100d.txt", encoding="utf-8", mode='r') as f:
    for line in f:
        values=line.split()
        # get the word
        word=values[0]
        if word in word2idx.keys():
            # get the vector
            vector = np.asarray(values[1:], 'float32')
            embedding_dict[word] = vector


In [44]:
#embedding_dict['debt']

In [45]:
num_words = len(word2idx) + 1

In [46]:
vocab_size = len(word2idx) + 1
output_size = 1
embedding_dim = 100

In [47]:
from tqdm import tqdm

In [48]:
embedding_matrix=np.zeros((num_words, embedding_dim))
import tqdm

In [49]:
for word,i in tqdm.tqdm(word2idx.items()):
    if i < num_words:
        vect=embedding_dict.get(word, [])
        if len(vect)>0:
            embedding_matrix[i] = vect[:embedding_dim]
        
        if word == '_UNK':
            embedding_matrix[i] = np.ones((1, embedding_dim))
           

100%|██████████| 12391/12391 [00:00<00:00, 318568.73it/s]


In [50]:
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import torch
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn


In [51]:
train_data = TensorDataset(torch.from_numpy(desc_words_tr), torch.from_numpy(train_num_data), torch.from_numpy(train_y))

In [52]:
batch_size = 32

train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size, drop_last=True)

In [53]:
class Sent_Ana(nn.Module):
    
    def __init__( self, vocab_size, embedding_dim, hidden_dim, output_size, n_layers, seq_len, num_size, drop_prob = 0.2 ):
        super(Sent_Ana, self).__init__()
        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        
        self.embedding = nn.Embedding( vocab_size, embedding_dim )
        self.gru = nn.GRU( embedding_dim, hidden_dim, n_layers, dropout = drop_prob, batch_first=True )
        self.dropout = nn.Dropout(drop_prob)
        self.fc1 = nn.Linear( hidden_dim, output_size )
        self.fc2 = nn.Linear( num_size, output_size )
        self.sigmoid = nn.Sigmoid()
        #self.fc2 = nn.Linear( hidden_dim2, output_size )
        
    def forward(self, x1, x2):
        #batch_size = x.size(0)
        x1 = x1.long()
        x1 = self.embedding(x1)
        #x = self.lstm( x )
        x1, states = self.gru(x1)
        x1 = x1.contiguous().view(-1, self.hidden_dim)
        x1 = self.dropout(x1)
        x1 = self.fc1( x1 )
        #x = F.relu( x )
        #x = self.fc2( x )
        x2 = self.fc2( x2 )
        x = torch.cat( (x1, x2), dim = 0 )
        x = torch.sigmoid( x )
        x = x.view(batch_size, -1)
        x = x[:,-1]
        #x = x.view(batch_size, -1)
        return x
    
    

In [54]:
vocab_size = len(word2idx) + 1
output_size = 1
embedding_dim = 100
hidden_dim = 32
n_layers = 2
hidden_dim2 = 16
num_size = 34

In [55]:
model = Sent_Ana( vocab_size, embedding_dim, hidden_dim,  output_size, n_layers, seq_len, num_size, drop_prob = 0.2 )


In [56]:
model.train()

Sent_Ana(
  (embedding): Embedding(12392, 100)
  (gru): GRU(100, 32, num_layers=2, batch_first=True, dropout=0.2)
  (dropout): Dropout(p=0.2, inplace=False)
  (fc1): Linear(in_features=32, out_features=1, bias=True)
  (fc2): Linear(in_features=34, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)

In [57]:
model.embedding.weight.data = torch.from_numpy( embedding_matrix ).float()
model.embedding.weight.requires_grad = True
model.embedding.weight.requires_grad

True

In [58]:
optimizer_fit = torch.optim.Adam(model.parameters(), lr=0.001)

In [59]:
criterion = torch.nn.BCELoss()

In [60]:
LOSS = {}
LOSS['training data'] = []
LOSS['validation data'] = []

In [61]:
epochs = 10

def train_model(epochs):
    
    for epoch in range(epochs):
        for x1, x2, y in train_loader:
            y = torch.tensor(y, dtype=torch.float)
            optimizer_fit.zero_grad()
            x1 = torch.tensor(x1, dtype=torch.float)
            x2 = torch.tensor(x2, dtype=torch.float)
            yhat = model(x1, x2)
            #print( y.shape )
            #print( yhat.shape )
            #print( x.shape )
            loss = criterion(yhat, y)
            optimizer_fit.zero_grad()
            loss.backward()
            optimizer_fit.step()
        
        print('Epoch: {} and Loss: {}'.format(epoch, loss.item()))
        LOSS['training data'].append(loss.item())
 
        
train_model(epochs)

C:\Users\Tsenguunn\AppData\Local\Temp\ipykernel_179936\2082474432.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y = torch.tensor(y, dtype=torch.float)
C:\Users\Tsenguunn\AppData\Local\Temp\ipykernel_179936\2082474432.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x1 = torch.tensor(x1, dtype=torch.float)
C:\Users\Tsenguunn\AppData\Local\Temp\ipykernel_179936\2082474432.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x2 = torch.tensor(x2, dtype=torch.float)


Epoch: 0 and Loss: 3.5038347244262695
Epoch: 1 and Loss: 3.3759231567382812
Epoch: 2 and Loss: 3.7900702953338623
Epoch: 3 and Loss: 3.395014524459839
Epoch: 4 and Loss: 0.41588208079338074
Epoch: 5 and Loss: 3.410327672958374
Epoch: 6 and Loss: 3.5040171146392822
Epoch: 7 and Loss: 3.4387919902801514
Epoch: 8 and Loss: 0.39426344633102417
Epoch: 9 and Loss: 0.3638418912887573


In [62]:
val_dat.columns

Index(['loan_amnt', 'funded_amnt', 'installment', 'annual_inc', 'desc',
       'delinq_2yrs', 'total_pymnt', 'recoveries', 'last_pymnt_amnt',
       'acc_now_delinq', 'chargeoff_within_12_mths', 'delinq_amnt', 'y_hat',
       'term_ 36 months', 'term_ 60 months', 'int_rate_num',
       'emp_length_1 year', 'emp_length_10+ years', 'emp_length_2 years',
       'emp_length_3 years', 'emp_length_4 years', 'emp_length_5 years',
       'emp_length_6 years', 'emp_length_7 years', 'emp_length_8 years',
       'emp_length_9 years', 'emp_length_< 1 year', 'home_ownership_MORTGAGE',
       'home_ownership_NONE', 'home_ownership_OTHER', 'home_ownership_OWN',
       'home_ownership_RENT', 'verification_status_Not Verified',
       'verification_status_Source Verified', 'verification_status_Verified'],
      dtype='object')

In [63]:
len_val = val_dat.shape[0]
#words = Counter()
val_words = []

In [64]:
val_dat.reset_index(drop=True, inplace=True)

In [65]:
for i in range(len_val):
    
    
    token_list = []
    ind = -1
    
    val1 = nlp(val_dat['desc'][i])
    
    for i, token in enumerate(val1):
        if i >= 5:
            tmp = token.text
            tmp = tmp.replace("<br", "")
            tmp = tmp.replace("br/", "")
            tmp = tmp.replace("/", "")
            tmp = tmp.replace(">", "")
            tmp = tmp.replace("%", "")
            if ( not is_number(tmp) ) and (tmp != '') and (tmp != ' ') and (tmp != '  ') and (tmp != '<') and (tmp != 'br') and (tmp != '/') and (tmp != '(') and (tmp != ')') and (tmp != '&') and (tmp != '*'):
            
                if tmp == "Borrower":
                    ind = i + 3
                if  i > ind:
                    if (tmp[-1] == ".") and (len(tmp) >=2):
                        token_list.extend( [tmp[:-1].lower(), tmp[-1]] )
                        #word = tmp[:-1]
                        #words.update( [word.lower()] )
                    else:
                        token_list.append( tmp.lower() )
                        #word = tmp
                        #words.update( [word.lower()] )
                        
    val_words.append(token_list)

In [66]:
for i, sentence in enumerate(val_words):
    # Looking up the mapping dictionary and assigning the index to the respective words
    val_words[i] = [word2idx[word] if word in word2idx else 1 for word in sentence]
    val_dat.loc[i, 'desc_len'] = len(sentence)

C:\Users\Tsenguunn\AppData\Local\Temp\ipykernel_179936\1704610499.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  val_dat.loc[i, 'desc_len'] = len(sentence)


In [67]:
val_words = pad_input(val_words, seq_len)

In [68]:
type(val_words)

numpy.ndarray

In [69]:
val_words.shape

(4218, 300)

In [70]:
val_y = np.array( val_dat['y_hat'].values )

In [71]:
val_dat.drop( columns=['y_hat'], inplace=True )

C:\Users\Tsenguunn\AppData\Local\Temp\ipykernel_179936\319868131.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  val_dat.drop( columns=['y_hat'], inplace=True )


In [72]:
val_words[2]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   

In [73]:
val_dat.loc[2, 'desc']

'  Borrower added on 08/29/11 > I am going to pay off another loan that has a higher interest rate.<br/>'

In [74]:
[idx2word[ww] for ww in val_words[2]]

['_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 

In [75]:
val_dat.drop( columns=['desc'], inplace=True )

C:\Users\Tsenguunn\AppData\Local\Temp\ipykernel_179936\2714905428.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  val_dat.drop( columns=['desc'], inplace=True )


In [76]:
val_dat.dtypes

loan_amnt                              float64
funded_amnt                            float64
installment                            float64
annual_inc                             float64
delinq_2yrs                            float64
total_pymnt                            float64
recoveries                             float64
last_pymnt_amnt                        float64
acc_now_delinq                         float64
chargeoff_within_12_mths               float64
delinq_amnt                            float64
term_ 36 months                        float64
term_ 60 months                        float64
int_rate_num                           float64
emp_length_1 year                      float64
emp_length_10+ years                   float64
emp_length_2 years                     float64
emp_length_3 years                     float64
emp_length_4 years                     float64
emp_length_5 years                     float64
emp_length_6 years                     float64
emp_length_7 

In [79]:
validation_num_data.shape

(4218, 34)

In [78]:
validation_num_data = np.array(val_dat)

In [80]:
np.savetxt( 'val_words.csv', val_words, delimiter=',' )

In [81]:
np.savetxt( 'val_y.csv', val_y, delimiter=',' )

In [82]:
np.savetxt( 'validation_num_data.csv', validation_num_data, delimiter=',' )

In [83]:
validation_data = TensorDataset(torch.from_numpy(val_words), torch.from_numpy(validation_num_data), torch.from_numpy(val_y))

In [84]:
val_loader = DataLoader(validation_data, shuffle=True, batch_size=batch_size, drop_last=True)

In [85]:
model.eval()
num_correct = 0

for x1, x2, y in val_loader:
    
    y = torch.tensor(y, dtype=torch.float)
    optimizer_fit.zero_grad()
    x1 = torch.tensor(x1, dtype=torch.float)
    x2 = torch.tensor(x2, dtype=torch.float)
    yhat = model(x1, x2)
    loss = criterion(yhat, y)
    LOSS['validation data'].append(loss.item())
    pred = torch.round(yhat.squeeze())  # Rounds the output to 0/1
    correct_tensor = pred.eq(y.float().view_as(pred))
    correct = np.squeeze(correct_tensor.cpu().numpy())
    num_correct += np.sum(correct)

print("Test loss: {:.3f}".format(np.mean(LOSS['validation data'])))
val_acc = num_correct/len(val_loader.dataset)
print("Test accuracy: {:.3f}%".format(val_acc*100))

C:\Users\Tsenguunn\AppData\Local\Temp\ipykernel_179936\4007162349.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y = torch.tensor(y, dtype=torch.float)
C:\Users\Tsenguunn\AppData\Local\Temp\ipykernel_179936\4007162349.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x1 = torch.tensor(x1, dtype=torch.float)
C:\Users\Tsenguunn\AppData\Local\Temp\ipykernel_179936\4007162349.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x2 = torch.tensor(x2, dtype=torch.float)


Test loss: 3.270
Test accuracy: 82.243%


In [ ]:
# Pre-trained embedding weights - requried_grad = False
# Test loss: 0.870
# Test accuracy: 84.116%